# <center>  Python for biologists
    
## Homework 10, Pandas



## Правила

- **Не создавайте доп. веток в репозитории, не копируйте ноутбук.**
- Пишите код только там, где это от вас требуется (`# YOUR CODE HERE`)
- Написав, удаляйте `raise NotImplementedError() `
- Не переименовывайте переменные, если они уже созданы. Происходят автопроверки.
- Под заданием для вас приведены проверки с помощью `assert`. Они падают с ошибкой если решено не правильно и молчат если всё правильно.
- Перед отправкой проверьте что ваш ноутбук работает (юпитер-ноутбук). Нажмите на панели сверху `Kernel` -> `Restart Kernel and Run All Cells...`. Проверье что нет ошибок.
    

#### Погнали!

---


In [1]:
import hashlib
from IPython.display import display

import numpy as np
import pandas as pd


pd.DataFrame.__hash__ = lambda self: int(
    hashlib.sha256(pd.util.hash_pandas_object(self, index=True).values).hexdigest(),
    base=16,
)
pd.core.indexes.base.Index.__hash__ = lambda self: int(
    hashlib.sha256(pd.util.hash_pandas_object(self, index=True).values).hexdigest(),
    base=16,
)
pd.Series.__hash__ = lambda self: int(
    hashlib.sha256(pd.util.hash_pandas_object(self, index=True).values).hexdigest(),
    base=16,
)

    
## I. Обзор дата-фреймов
    
Здесь будем работать с дата-фреймом `olympics`



<a href='https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results'><img src="https://i.ytimg.com/vi/b4wlPz9p4e8/maxresdefault.jpg" align="center" width="500" ></a>

In [138]:
olympics = pd.read_csv('https://github.com/cleuer/olympics/raw/master/src/main/resources/olympic_history_120_years.csv')

#### **Задание 1**💅

Выведите первые 3 строки дата-фрейма

In [95]:
result = olympics.head(3)
display(result)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [4]:
assert len(result) == 3
assert isinstance(result, pd.core.frame.DataFrame)
assert hash(result) == 507558853293347520

#### **Задание 2** 💅

Выведите названия всех колонок в дата-фрейме

In [96]:
result = olympics.columns
display(result)

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [6]:
assert len(result) == 15
assert isinstance(result, pd.core.indexes.base.Index)
assert hash(result) == 204161812668953242

#### **Задание 3** 💅

Сколько колонок имеют `ei` в имени?

In [97]:
result = sum('ei' in col for col in olympics.columns)

In [8]:
assert result == 2

#### **Задание 4** 💅

Выведите размерность дата-фрейма

In [98]:
result = olympics.shape

In [10]:
assert result == (271116, 15)

#### **Задание 5** 💅

Выведите количество NA в колонке `Weight`

In [99]:
result = olympics['Weight'].isna().sum()

In [12]:
assert result == 62875

#### **Задание 6** 💅

Выведите количество уникальных значений в колонке `Weight` (для этого есть отдельный метод)

In [100]:
result = olympics['Weight'].nunique()

In [14]:
assert result == 220

#### **Задание 7** 💅

Выведите самое часто-встречаемое значение в колонке `Weight`

In [101]:
result = olympics['Weight'].mode()[0]

In [16]:
assert result == 70.0

#### **Задание 8** 💅

Выведите максимальное значение в колонке `Weight`

In [102]:
result = olympics['Weight'].max()

In [21]:
assert result == 214.0

#### **Задание 9** 💅

Сколько колонок в дата-фрейме имеют тип данных `float64`?

In [103]:
result = (olympics.dtypes == 'float64').sum()

In [23]:
assert result == 3

## II. Отбор данных

#### **Задание 10** 💅

Как зовут атлета в `140228`й по счету строке дата-фрейма?

In [125]:
result = olympics.iloc[140227]['Name']

'Yuliya Vyacheslavovna Lipnitskaya'

In [25]:
assert result == 'Yuliya Vyacheslavovna Lipnitskaya'

#### **Задание 11** 💅

В каком городе проходили игры, запись с которых сделана в строке с индексом 360?

In [105]:
result = olympics.iloc[360]['City']

In [29]:
assert result == 'Seoul'

#### **Задание 12** 💅

Отберите все записи относящиеся к мужчинам

In [106]:
result = olympics[olympics['Sex'] == 'M']

In [31]:
assert len(result) == 196594
assert hash(result) == 53408147431013694

#### **Задание 13** 💅

Отберите все записи относящиеся летним играм в Барселоне

In [107]:
result = olympics[(olympics['City'] == 'Barcelona') & (olympics['Season'] == 'Summer')]

In [33]:
assert len(result) == 12977
assert hash(result) == 1257297863566133861

#### **Задание 14** 💅

Какую медаль получила Юлия Липницкая в зимних играх 2014 года в индивидуальном зачёте?

In [139]:
result = olympics[
    (olympics['Name'].str.contains('Lipnitskaya', case=False)) &
    (olympics['Year'] == 2014) &
    (olympics['Season'] == 'Winter') &
    (olympics['Event'].str.contains('Singles', case=False))
]['Medal']
print(result)
# Почему оно не принимает? У нее же записано, что в индивидуальном зачете NaN

140226    NaN
Name: Medal, dtype: object


In [137]:
assert result is np.nan

AssertionError: ignored

#### **Задание 15** 💅

Замените все NA в колонке `Medal` на `"No medal"` (для этого есть специальный метод)

In [135]:
olympics['Medal'] = olympics['Medal'].fillna('No medal')

In [54]:
assert olympics.Medal.isna().sum() == 0
assert olympics.Medal.value_counts().to_dict()['No medal'] == 231333
assert hash(olympics) == 1196581486896556999

#### **Задание 16** 💅

Создайте в дата-фрейме колонку `BMI` с соотвествующими значениями

In [55]:
olympics['BMI'] = olympics['Weight'] / (olympics['Height'] / 100) ** 2

In [56]:
assert np.isclose(olympics.BMI.mean(), 22.7845, atol=1e-3)
assert np.isclose(olympics.BMI.std(), 2.9120, atol=1e-3)
assert olympics.BMI.isna().sum() == 64263
assert hash(olympics) == 1059869394349329045

**Доп. задание**

NA (Not Available): термин часто используется в статистике и анализе данных. Он указывает на отсутствие данных в ячейке или переменной. NA широко используется в языках программирования, таких как R, для обозначения отсутствия данных.

None: специфический термин, используемый в некоторых языках программирования, таких как Python. None представляет отсутствие значения. Он отличается от нуля, пустой строки или других типичных "пустых" значений в том, что он явно указывает на отсутствие значения.

NaN (Not a Number):  термин используется преимущественно в численном анализе. NaN является стандартным способом представления неопределенных или недопустимых числовых результатов во многих системах и языках программирования, включая Python (в библиотеках, таких как NumPy и Pandas) и JavaScript. Примерами операций, которые могут привести к NaN, являются деление нуля на ноль или извлечение квадратного корня из отрицательного числа в контексте вещественных чисел.

NULL: термин чаще всего встречается в базах данных и языках запросов, таких как SQL. NULL указывает на отсутствие значения в ячейке базы данных. Он отличается от пустой строки или нуля, поскольку эти значения все еще считаются конкретными значениями, в то время как NULL означает полное отсутствие данных.

### **Задание XXX**

Нажмите на панели сверху `Kernel` -> `Restart Kernel and Run All Cells...`. Проверьте что нет ошибок.

---
# EDA

В 2 части этого ДЗ вам предстоит написать ваш модуль для EDA. Покажите тут, что у вас получилось!

> По идее, вам нужно только запустить ячейку и всё. Но можете изменить датасет если хотите.

>Приветствие!

>Количество наблюдений (строк) и параметров (признаков, столбцов)
Какие типы данных у каждой из колонок?
Какие признаки численные, какие строковые, а какие - факторные (категориальные)? Это не всегда можно сделать на основе типов данных. Категориальный признак может быть закодирован несколькими цифрами (0, 1, 2), может быть закодирован буквами ("Gold", "Silver"). С другой стороны рост - признак численный, но в вашем df могут быть люди только с ростом 1 и 2 метра. Ну и понятно, что имена людей - признак строковый, но не категориальный. Таким образом автоматическое разделение на типы признаков без ручной проверки всегда имеет погрешность, это не страшно. Просто придумайте для вашего модуля какое-нибудь более-менее рациональное decision rule. Сохраните названия численных, строковых, и категориальных колонок в коллекции, напечатайте их, но не возвращайте. Они понадобятся далее.

>По каждому категориальному признаку выведите counts и frequences (counts / total_counts) значений в df

>По каждому численному признаку выведете его min, max, mean, std, q0.25, median, q.075

>Выведите количество выбросов по каждому численному признаку, в котором есть выбрсы (здесь возьмем правило ± 1.5 IQR)

>Сколько пропущенных значений всего в df, сколько строк содержат пропущенные значения, какие столбцы содержат пропущенные значения?

>Сколько дублирующихся строк в df?

In [146]:
# Можете вставить любой репрезантивный дата-фрейм на ваше усмотрение :)
data_dir = 'https://raw.githubusercontent.com/Python-BI-2023/course_materials/main/data/'
df = pd.read_csv(data_dir + 'titanic.csv')

In [148]:
from my_awesome_eda import run_eda

run_eda(df)

Welcome to the EDA of your DataFrame!
Number of observations (rows): 418
Number of features (columns): 11
Data types of each column:
PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
Numeric columns: Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Categorical columns: Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
Numeric columns: Index(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Categorical columns: Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Pclass'], dtype='object')
Numeric columns: Index(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Categorical columns: Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Pclass'], dtype='object')
Numeric columns: Index(['Passe

---

Надеюсь, у тебя всё получилось! Но в любом случае - ты молодец:)🥳

---